## Download country Overture buildings

Download buildings for the 5 countries from overture maps

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

In [2]:
country_names = ['Germany', 'Poland', 'Czechia', 'Slovakia', 'Austria']

In [3]:
regions_datadir = "/data/uscuni-eurofab-overture/"
char_type = 'unweighted_lag'

Download a shapefile of all EU countries, if not present.

In [4]:
# download countries
# !wget https://gisco-services.ec.europa.eu/distribution/v2/countries/geojson/CNTR_RG_01M_2024_3035.geojson

In [5]:
countries = gpd.read_file('CNTR_RG_01M_2024_3035.geojson').to_crs(epsg=3035)
country_polygons = countries[countries['NAME_ENGL'].isin(country_names)]
country_polygons = country_polygons.explode()
country_polygons['area'] = country_polygons.area
country_polygons = country_polygons.sort_values('area', ascending=False).iloc[:5].reset_index(drop=True).to_crs(epsg=4236)

In [6]:
from core.generate_streets import record_batch_reader

In [7]:
%%time

for _, country in country_polygons.iterrows():
    
    print(country.NAME_ENGL.lower())
    
    batches = record_batch_reader('building', country.geometry.bounds).read_all()
    gdf = gpd.GeoDataFrame.from_arrow(batches)
    gdf = gdf.iloc[gdf.sindex.query(country.geometry, predicate='intersects')]
    gdf['source'] = gdf['sources'].str[0].str['dataset']
    gdf.to_parquet(f'/data/uscuni-eurofab-overture/overture_buildings/ov_{country.NAME_ENGL.lower()}.pq')

germany
poland
austria
czechia
slovakia
CPU times: user 11min 10s, sys: 2min 14s, total: 13min 25s
Wall time: 1h 54min 16s


Merge all data together for regional deliniations

In [7]:
all_gdf = []
for  _, country in country_polygons.iterrows():
    all_gdf.append(
        gpd.read_parquet(
            f'/data/uscuni-eurofab-overture/overture_buildings/ov_{country.NAME_ENGL.lower()}.pq',
                                    columns=['id', 'geometry', 'source', 'class']
        ).set_crs(epsg=4326).to_crs(epsg=3035)
    )
all_gdf = pd.concat(all_gdf, ignore_index=True)

In [8]:
all_gdf.shape

(81059797, 4)

In [9]:
all_gdf.head()

,id,geometry,source,class
0,08b1f883b3908fff0200da4a7ab55d9c,"POLYGON ((4416686.135 2699131.933, 4416692.287...",Microsoft ML Buildings,None
1,08b1f883a2144fff02008edf67e9e9ff,"POLYGON ((4419272.063 2702804.844, 4419271.397...",OpenStreetMap,None
2,08b1f883a2144fff02002871fda26c5a,"POLYGON ((4419275.986 2702805.046, 4419279.539...",OpenStreetMap,hut
3,08b1f883a0d8bfff020041f439a51c5c,"POLYGON ((4419086.837 2704704.765, 4419091.894...",OpenStreetMap,retail
4,08b1f883a2133fff02005f2912e4c742,"POLYGON ((4418843.294 2702688.792, 4418841.541...",OpenStreetMap,None


In [10]:
all_gdf.to_parquet(f'/data/uscuni-eurofab-overture/overture_buildings/ov_ce.pq')

## Define regions on OV data

Split the whole Central Europe to sub-regions based on their morphological continuity.

In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN

In [2]:
gdf = gpd.read_parquet('/data/uscuni-eurofab-overture/overture_buildings/ov_ce.pq')

Aggregate the building centroids to 100x100m grid and use count as weight for DBSCAN.

In [3]:
cents = gdf.centroid
gdf['x'], gdf['y'] = cents.x, cents.y
gdf['id'] = gdf.index.values
data = gdf[["x", "y", 'id']]

In [4]:
data[["x_100", "y_100"]] = np.around(data[["x", "y"]], decimals=-2)
grid = data[["id", "x_100", "y_100"]].groupby(["x_100", "y_100"]).count().reset_index()

/tmp/ipykernel_24138/4289293720.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[["x_100", "y_100"]] = np.around(data[["x", "y"]], decimals=-2)
/tmp/ipykernel_24138/4289293720.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[["x_100", "y_100"]] = np.around(data[["x", "y"]], decimals=-2)


Cluster the grid cells in DBSCAN with 400 meters distance. Equivalent to  400- ~550 distance between buildings.

In [5]:
dbscan = DBSCAN(400, n_jobs=-1).fit(grid[["x_100", "y_100"]], sample_weight=grid["id"])
grid["labels"] = dbscan.labels_

In [6]:
data = pd.merge(data, grid, "left", on=["x_100", "y_100"])

Identify core urban areas (with more than 10k buildings within a single cluster).

In [7]:
counts = data.labels.value_counts()
data["core"] = data.labels.isin(counts[counts > 10000].index.drop(-1))

In [8]:
cores = data[data.core]

Assign each non-core cluster and non-core building without a cluster to the nearest cluster. Done using the grid for efficiency.

In [9]:
grid["core"] = grid.labels.isin(counts[counts > 10000].index.drop(-1))
grid_cores = grid[grid.core]
grid_cores = gpd.GeoDataFrame(
    grid_cores["labels"],
    geometry=gpd.points_from_xy(grid_cores["x_100"], grid_cores["y_100"]),
    crs=3035,
)
grid_cores_dissolved = grid_cores.dissolve("labels")

In [10]:
grid_non_cores = grid[~grid.core]
grid_non_cores = gpd.GeoDataFrame(
    grid_non_cores["labels"],
    geometry=gpd.points_from_xy(grid_non_cores["x_100"], grid_non_cores["y_100"]),
    crs=3035,
)

In [11]:
grid_non_cores_clustered = grid_non_cores[grid_non_cores.labels != -1]
grid_non_cores_outliers = grid_non_cores[grid_non_cores.labels == -1]

In [12]:
grid_non_cores_clustered_dissolved = grid_non_cores_clustered.dissolve("labels")

In [13]:
%%time
nearest = grid_cores.sindex.nearest(
    grid_non_cores_clustered_dissolved.geometry, return_all=False
)

CPU times: user 23.5 s, sys: 122 ms, total: 23.6 s
Wall time: 23.6 s


In [14]:
grid_non_cores_clustered_dissolved["nearest_core"] = grid_cores.labels.values[
    nearest[1]
]

In [15]:
%%time
nearest_outliers = grid_cores.sindex.nearest(
    grid_non_cores_outliers.geometry, return_all=False
)

CPU times: user 3.23 s, sys: 0 ns, total: 3.23 s
Wall time: 3.23 s


Finalise and Create final region label.

In [16]:
grid_non_cores_outliers["nearest_core"] = grid_cores.labels.values[nearest_outliers[1]]

/home/krasen/morphometrics/.pixi/envs/default/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [17]:
grid_non_cores = pd.concat(
    [
        grid_non_cores_clustered_dissolved.reset_index().explode(ignore_index=True),
        grid_non_cores_outliers,
    ],
    ignore_index=True,
)

In [18]:
grid_non_cores["x_100"] = grid_non_cores.geometry.x
grid_non_cores["y_100"] = grid_non_cores.geometry.y

In [19]:
data = pd.merge(
    data,
    grid_non_cores[["x_100", "y_100", "nearest_core"]],
    "left",
    on=["x_100", "y_100"],
)

In [20]:
data["region"] = data.labels
data.loc[~data.core, "region"] = data.loc[~data.core, "nearest_core"]

In [21]:
data = data.rename(
    columns={
        "id_x": "id",
        "id_y": "weight",
        "labels": "dbscan_cluster",
    }
)

In [22]:
data

,x,y,id,x_100,y_100,weight,dbscan_cluster,core,nearest_core,region
0,4.416687e+06,2.699138e+06,0,4416700.0,2699100.0,1,37069,False,34737.0,34737
1,4.419276e+06,2.702800e+06,1,4419300.0,2702800.0,2,-1,False,34737.0,34737
2,4.419278e+06,2.702806e+06,2,4419300.0,2702800.0,2,-1,False,34737.0,34737
3,4.419094e+06,2.704702e+06,3,4419100.0,2704700.0,1,-1,False,34737.0,34737
4,4.418845e+06,2.702684e+06,4,4418800.0,2702700.0,9,37792,False,34737.0,34737
...,...,...,...,...,...,...,...,...,...,...
81059792,4.955654e+06,2.970479e+06,81059792,4955700.0,2970500.0,1,104689,True,NaN,104689
81059793,4.955637e+06,2.970488e+06,81059793,4955600.0,2970500.0,1,104689,True,NaN,104689
81059794,4.958197e+06,2.970401e+06,81059794,4958200.0,2970400.0,3,104689,True,NaN,104689
81059795,4.958187e+06,2.970410e+06,81059795,4958200.0,2970400.0,3,104689,True,NaN,104689


Save data

In [23]:
pd.concat(
    [
        grid_cores,
        grid_non_cores[["nearest_core", "geometry"]].rename(
            columns={"nearest_core": "labels"}
        ),
    ]
).dissolve("labels").convex_hull.to_frame("convex_hull").to_parquet(
    "/data/uscuni-eurofab-overture/regions/ov_ce_region_hulls.parquet"
) 

In [24]:
data.to_parquet("/data/uscuni-eurofab-overture/regions/ov_id_to_region.parquet")

Split all buildings into regions.

In [25]:
%%time
for region_id, group in data.groupby('region'):

    region_id = int(region_id)
    
    buildings = gdf.iloc[group.id]
    buildings['iid'] = buildings.index.values
    buildings.to_parquet(f'/data/uscuni-eurofab-overture/regions/buildings/buildings_{region_id}.pq')

/home/krasen/morphometrics/.pixi/envs/default/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/krasen/morphometrics/.pixi/envs/default/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/krasen/morphometrics/.pixi/envs/default/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is

CPU times: user 1min 6s, sys: 8.78 s, total: 1min 15s
Wall time: 1min 15s


/home/krasen/morphometrics/.pixi/envs/default/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [28]:
# gpd.read_parquet("/data/uscuni-eurofab-overture/regions/ov_ce_region_hulls.parquet")

## Download overture streets

Download and store overture streets, based on the generated regions.

In [8]:
from core.generate_streets import read_overture_region_streets

import gc
import glob

import geopandas as gpd
import momepy as mm
import numpy as np
import pandas as pd
import shapely
from libpysal.graph import Graph, read_parquet
import datetime

In [9]:
streets_dir = '/data/uscuni-eurofab-overture/overture_streets/'
regions_datadir = "/data/uscuni-eurofab-overture/"

In [10]:
region_hulls = gpd.read_parquet(
        regions_datadir + "regions/" + "ov_ce_region_hulls.parquet"
    )
region_hulls.shape

(709, 1)

In [11]:
def process_and_save(region_hull, region_id):
    print('Processing', region_id, datetime.datetime.now())
    streets = read_overture_region_streets(region_hull, region_id)
    streets.to_parquet(streets_dir + f'streets_{region_id}.pq')

In [12]:
downloaded_streets = glob.glob(streets_dir + '*')
downloaded_streets = [int(s.split('_')[-1][:-3]) for s in downloaded_streets]

region_hulls = region_hulls.loc[~region_hulls.index.isin(downloaded_streets), ]

region_hulls

In [18]:
%%capture cap

from joblib import Parallel, delayed

n_jobs = -1
new = Parallel(n_jobs=n_jobs)(
    delayed(process_and_save)(region_hull.iloc[0], region_id) for region_id, region_hull in region_hulls.to_crs(epsg=4326).iterrows()
)


Processing 7867 2025-02-12 14:46:56.938941
Processing 8160 2025-02-12 14:56:00.209572
Processing 6037 2025-02-12 14:46:56.878705
Processing 9871 2025-02-12 14:56:58.754359
Processing 12041 2025-02-12 15:07:17.587638
Processing 14496 2025-02-12 15:17:26.047437
Processing 6108 2025-02-12 14:46:56.877414
Processing 8360 2025-02-12 14:56:29.364147
Processing 10957 2025-02-12 15:06:21.678194
Processing 12493 2025-02-12 15:13:51.830129
Processing 7355 2025-02-12 14:46:56.893627
Processing 10078 2025-02-12 14:57:15.236743
Processing 12214 2025-02-12 15:07:30.175483
Processing 13633 2025-02-12 15:16:28.055702
Processing 16414 2025-02-12 15:26:14.876770
Processing 7674 2025-02-12 14:46:56.907937
Processing 9147 2025-02-12 14:56:36.248254
Processing 11944 2025-02-12 15:07:14.104998
Processing 13566 2025-02-12 15:16:26.017605
Processing 16439 2025-02-12 15:26:17.912310
Processing 20332 2025-02-12 15:35:01.923214
Processing 6612 2025-02-12 14:46:56.895746
Processing 9754 2025-02-12 14:56:57.361163

In [5]:
%%time

all_streets = []
for region_id, _ in region_hulls.to_crs(epsg=4326).iterrows():
    streets = gpd.read_parquet(streets_dir + f'streets_{region_id}.pq')
    all_streets.append(streets)
all_streets = pd.concat(all_streets)

CPU times: user 2min 8s, sys: 38.8 s, total: 2min 46s
Wall time: 2min 5s


In [8]:
all_streets.drop_duplicates('id').shape

(23332865, 21)